# Домашнее задание № 7

In [1]:
import re, random
import pandas as pd
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from string import punctuation
from collections import Counter
import numpy as np
from razdel import tokenize as razdel_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
stopwords_eng = set(stopwords.words("english"))


lemmatizer = WordNetLemmatizer()
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

### Задание 1 Реализовать алгоритм Леска и проверить его на реальном датасете (8 баллов)

Ворднет можно использовать для дизамбигуации. Самый простой алгоритм дизамбигуации - алгоритм Леска. В нём нужное значение слова находится через пересечение слов контекста, в котором употреблено это слово, с определениями значений слова из ворднета. Значение с максимальным пересечением - нужное.

Реализуйте его

In [2]:
def lesk(word, sentence):
    
    bestsense = 0
    maxoverlap = 0
    
    sentence = [lemmatizer.lemmatize(word) for word in sentence if word not in stopwords_eng]
    synsets = wn.synsets(word)
    
    for i, synset in enumerate(synsets):
        definition = synset.definition().lower().split()
        definition = [lemmatizer.lemmatize(word) for word in definition if word not in stopwords_eng]
        overlap = len(list(set(sentence) & set(definition)))
        
        if overlap > maxoverlap:
            maxoverlap = overlap
            bestsense = i
       
    return bestsense

Работать функция должна как-то так:

In [3]:
# на вход подается элемент результата работы уже написанной вами функции get_words_in_context
lesk('day', 'some point or period in time'.split()) # для примера контекст совпадает с одним из определений
# а на выходе индекс подходящего синсета

1

In [4]:
wn.synsets('day')[1].definition()

'some point or period in time'

In [5]:
wn.synsets('day')

[Synset('day.n.01'),
 Synset('day.n.02'),
 Synset('day.n.03'),
 Synset('day.n.04'),
 Synset('day.n.05'),
 Synset('day.n.06'),
 Synset('day.n.07'),
 Synset('sidereal_day.n.01'),
 Synset('day.n.09'),
 Synset('day.n.10')]

In [6]:
# с помощью этого индекса достаем нужный синсет
wn.synsets('day')[1].definition()

'some point or period in time'

In [7]:
wn.synsets('day')[1].examples()

['it should arrive any day now',
 'after that day she never trusted him again',
 'those were the days',
 'these days it is not unusual']

**Проверьте насколько хорошо работает такой метод на датасете из семинара**

Корпус состоит из предложений, где у каждого слова три поля - значение, лемма и само слово. Значение пустое, когда слово однозначное, а у многозначных слов стоит тэг вида **'long%3:00:02::'** Это тэг wordnet'ного формата

Вот так можно достать соответствующий этому тэгу синсет:

In [8]:
wn.lemma_from_key('review%2:31:00::').synset()

Synset('review.v.01')

Когда с помощью алгоритма Леска вы найдете подходящее значение, их можно просто сравнить

In [9]:
# wn.synsets('long')[i]

In [10]:
# тут подходит второй синсет (!в вашем решении вместо индекса единицы будет результат работы функции lesk)
wn.synsets('long')[1] == wn.lemma_from_key('long%3:00:02::').synset()

True

In [11]:
# можно сравнить со всеми, чтобы убедиться
[(syns, wn.synsets('long')[i] == wn.lemma_from_key('long%3:00:02::').synset()) 
  for i, syns in enumerate(wn.synsets('long'))]

[(Synset('hanker.v.01'), False),
 (Synset('long.a.01'), True),
 (Synset('long.a.02'), False),
 (Synset('long.s.03'), False),
 (Synset('retentive.a.01'), False),
 (Synset('long.a.05'), False),
 (Synset('long.a.06'), False),
 (Synset('long.s.07'), False),
 (Synset('farseeing.s.02'), False),
 (Synset('long.s.09'), False),
 (Synset('long.r.01'), False),
 (Synset('long.r.02'), False)]

**Вам нужно для каждого многозначного слова (т.е. у него есть тэг в первом поле) с помощью алгоритма Леска предсказать нужный синсет и сравнить с правильным. Посчитайте процент правильных предсказаний (accuracy).**

Если считается слишком долго, возьмите поменьше предложений (например, только тысячу)

In [12]:
with open('corpus_wsd_50k.txt') as file:
    corpus = file.read().split('\n\n')

In [13]:
corpus_wsd = []
for sent in corpus:
    corpus_wsd.append([s.split('\t') for s in sent.split('\n')])

In [14]:
corpus_wsd[0]

[['', 'how', 'How'],
 ['long%3:00:02::', 'long', 'long'],
 ['', 'have', 'has'],
 ['', 'it', 'it'],
 ['be%2:42:03::', 'be', 'been'],
 ['', 'since', 'since'],
 ['', 'you', 'you'],
 ['review%2:31:00::', 'review', 'reviewed'],
 ['', 'the', 'the'],
 ['objective%1:09:00::', 'objective', 'objectives'],
 ['', 'of', 'of'],
 ['', 'you', 'your'],
 ['benefit%1:21:00::', 'benefit', 'benefit'],
 ['', 'and', 'and'],
 ['service%1:04:07::', 'service', 'service'],
 ['program%1:09:01::', 'program', 'program'],
 ['', '?', '?']]

In [24]:
def test_lesk(n):
    accuracy = []
    total = 0
    correct = 0
    for sent in corpus_wsd:
        try:
            sent_tokens = [word[2].lower() for word in sent]
            for i, word in enumerate(sent):
                if word[0]:
                    total += 1
                    context = sent_tokens[max(0, i-5):i] + sent_tokens[i+1:i+5]
                    lesk = lesk(word[1], context)
                    if wn.synsets(word[1])[lesk] == wn.lemma_from_key(word[0]).synset():
                        correct += 1
        except IndexError:
            pass
    accuracy = correct/total
    return accuracy

In [16]:
test_lesk(corpus_wsd[:1000])

0.4687490882111432

### Задание 2* (2 балла)

В семинаре для WSI на данных Диалога использовался только Fastext. Попробуйте заменить его на адаграм (обучите свою модель или используйте предобученную out.pkl или https://s3.amazonaws.com/kostia.lopuhin/all.a010.p10.d300.w5.m100.nonorm.slim.joblib), а также поэкспериментируйте с разными алгоритмами кластеризации и их параметрами (можно использовать только те алгоритмы, которые обучаются достаточно быстро)

Для каждого эксперимента рассчитайте ARI

In [17]:
!pip install git+https://github.com/lopuhin/python-adagram.git

  Cloning https://github.com/lopuhin/python-adagram.git to /private/var/folders/78/prt6vf5x0yvd8c8qqcz2vz2m0000gn/T/pip-req-build-vt0ny7eo
  Running command git clone -q https://github.com/lopuhin/python-adagram.git /private/var/folders/78/prt6vf5x0yvd8c8qqcz2vz2m0000gn/T/pip-req-build-vt0ny7eo


In [18]:
import adagram

In [19]:
import re
from sklearn.cluster import AffinityPropagation, AgglomerativeClustering, DBSCAN, \
                            KMeans, MiniBatchKMeans, Birch, MeanShift, SpectralClustering
from sklearn.metrics import adjusted_rand_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn import metrics

In [20]:
df = pd.read_csv('train.csv', sep='\t')

In [21]:
df.head()

,context_id,word,gold_sense_id,predict_sense_id,positions,context
0,1,замок,1,NaN,"0-5, 339-344",замок владимира мономаха в любече . многочисле...
1,2,замок,1,NaN,"11-16, 17-22, 188-193","шильонский замок замок шильйон ( ) , известный..."
2,3,замок,1,NaN,299-304,проведения архитектурно - археологических рабо...
3,4,замок,1,NaN,111-116,"топи с . , л . белокуров легенда о завещании м..."
4,5,замок,1,NaN,"134-139, 262-267",великий князь литовский гедимин после успешной...


In [22]:
def normalize(text):
    
    words = [token.text.strip(punct) for token in list(razdel_tokenize(text))]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops_2]

    return word

In [23]:
vm = adagram.VectorModel.load("out.pkl")